In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as ply
import ipywidgets as iw
from oneibl import one
import brainbox.plot as bbp
from fitplot_funs import err_wght_sync
from export_funs import trialinfo_to_df

%matplotlib inline

fit_types = {'Stimulus on': 'stim',
             'Feedback': 'fdbck',
             'Prior estimate gain': 'prior',}

# Iterate though directories and find which mice/sessions we have fits for
mice = [x for x in os.listdir('./fits/') if os.path.isdir('./fits/' + x)]
fitsess = {mouse: [y[:-2] for y in os.listdir(f'./fits/{mouse}/') if y.split('.')[-1] == 'p'] for mouse in mice}

mousewidget = iw.Dropdown(options=mice)
sesswidget = iw.Dropdown()
cellwidget = iw.Dropdown()
fitwidget = iw.Dropdown()

def updatesess(*args):
    sesswidget.options = fitsess[mousewidget.value]
sesswidget.observe(updatesess)

mouse = 'ZM_2240'
sess = '2020-01-22_session_2020-04-12_probe0_fit.p'
probe_idx=0


fits = np.load(f'./fits/{mouse}/{sess}', allow_pickle=True)
wts_per_kern = fits['wts_per_kern']
kern_length = fits['kern_length']
binw = fits['glm_binsize']
uuid = fits['session_uuid']
p_est = fits['prior_est']
trdf = trialinfo_to_df(uuid, maxlen=2.)
one = one.ONE()
spikes = one.load(uuid, ['spikes.times'])[probe_idx]
clu = one.load(uuid, ['spikes.clusters'])[probe_idx]

Connected to https://alyx.internationalbrainlab.org as berk.gercek
Connected to https://alyx.internationalbrainlab.org as berk.gercek


In [24]:
percentile = 90 # We should only show cells whose prior gain / gain estimate variance is above this percentile
fullfits = fits['fits']
fitdf = fullfits[np.isfinite(fullfits.prior)]
prior_threshold = np.percentile(np.abs(fitdf.prior / fitdf.varprior), percentile)
subsetdf = fitdf[np.abs(fitdf.prior / fitdf.varprior) > prior_threshold]
if 'bias' not in trdf.columns:
    trdf = trdf.join(p_est)
trdf = trdf[np.isfinite(trdf.bias)]

In [25]:
SMALLER_SIZE = 16
SMALL_SIZE = 20
MEDIUM_SIZE = 24
BIGGER_SIZE = 32

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

@iw.interact
def plot_kern(cell=subsetdf.index.to_list(), kern=fit_types.keys()):
    fig, axes = plt.subplots(2, 1, figsize=(15, 9))
    currfit = fitdf.loc[cell]
    timescale = np.arange(0, kern_length, binw)
    if fit_types[kern] == 'stim':
        weight1 = currfit['stim_L']
        weight2 = currfit['stim_R']
        err1 = err_wght_sync(currfit['varstim_L'], weight1)
        err2 = err_wght_sync(currfit['varstim_R'], weight2)
        label1 = 'Left stimulus onset'
        label2 = 'Right stimulus onset'
        title = 'Kernels fit to right and left stimulus onset'
        event_t1 = trdf[np.isfinite(trdf.contrastLeft)].stimOn_times
        event_t2 = trdf[np.isfinite(trdf.contrastRight)].stimOn_times
    elif fit_types[kern] == 'fdbck':
        weight1 = currfit['fdbck_corr']
        weight2 = currfit['fdbck_incorr']
        err1 = err_wght_sync(currfit['varfdbck_corr'], weight1)
        err2 = err_wght_sync(currfit['varfdbck_incorr'], weight2)
        label1 = 'Correct feedback'
        label2 = 'Incorrect feedback'
        title = 'Kernels fit to correct and incorrect feedback'
        event_t1 = trdf[trdf.feedbackType == 1].stimOn_times
        event_t2 = trdf[trdf.feedbackType == -1].stimOn_times
    elif fit_types[kern] == 'prior':
        plt.close(fig)
        fig, axes = plt.subplots(1, 1, figsize=(15, 9))
        n_perc = 10
        cmap = sns.color_palette('GnBu_d', n_perc)
        percentiles = np.linspace(0, 100, n_perc + 1)
        percvals = np.percentile(trdf.bias, percentiles)
        print('Gain modulation of prior estimate:', currfit['prior'], 'Std dev of fit:', currfit['varprior'])
        for i in range(1, n_perc + 1):
            perctrials = trdf[(percvals[i-1] < trdf.bias) & (trdf.bias < percvals[i])].feedback_times
            oldmin, oldmax = axes.get_ylim()
            bbp.peri_event_time_histogram(spikes, clu, perctrials, int(cell[4:]), t_before=0.4, t_after=0.6, ax=axes,
                                          error_bars='sem',
                                          pethline_kwargs={'color': cmap[i-1], 'lw': 2, 'label': f'{percentiles[i-1]} to {percentiles[i]} percentile psy est'},
                                          errbar_kwargs={'color': cmap[i-1], 'alpha': 0.2})
            newmin, newmax = axes.get_ylim()
            axes.set_ylim([min((oldmin, newmin)), max((newmax, oldmax))])
            axes.set_yticks(axes.get_ylim())

        axes.set_title('PSTH about stimulus for percentiles of psytrack estimate\n'
                       'GLM gain term: {:.2f}, Std. dev. of term: {:.2f}'.format(currfit['prior'], currfit['varprior']))
        axes.set_xlabel('Time (s)')
        axes.set_ylabel('Firing rate')
        axes.legend()
        plt.tight_layout()
        return
    bbp.peri_event_time_histogram(spikes, clu, event_t1, int(cell[4:]), t_before=0., t_after=0.6, ax=axes[0],
                                  error_bars='sem',
                                  pethline_kwargs={'color': 'navy', 'lw': 2, 'label': label1 + ' PSTH'},
                                  errbar_kwargs={'color': 'navy', 'alpha': 0.2})
    ymin1, ymax1 = axes[0].get_yticks()[1:]
    bbp.peri_event_time_histogram(spikes, clu, event_t2, int(cell[4:]), t_before=0., t_after=0.6, ax=axes[0],
                                  error_bars='sem',
                                  pethline_kwargs={'color': 'orange', 'lw': 2, 'label': label2 + ' PSTH'},
                                  errbar_kwargs={'color': 'orange', 'alpha': 0.2})
    ymin2, ymax2 = axes[0].get_yticks()[1:]
    axes[0].set_ylim([min((ymin1, ymin2)), max((ymax1, ymax2))])
    axes[0].legend()
    
    axes[0].set_title(f'PSTH about {kern}')
    plt.autoscale(axes[0])
    axes[1].errorbar(timescale, weight1, yerr=err1, label=label1)
    axes[1].errorbar(timescale, weight2, yerr=err2, label=label2)
    axes[1].legend()
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Kernel value')
    axes[1].set_title(title)
    plt.tight_layout()
    # Next: Tuning curve against psytrack estimate value (Cell 143), (cell 164)
    

interactive(children=(Dropdown(description='cell', options=('cell8', 'cell28', 'cell108', 'cell119', 'cell123'…

In [21]:
trdf

,choice,response_times,probabilityLeft,feedbackType,feedback_times,contrastLeft,contrastRight,goCue_times,stimOn_times,bias
1,1,21.219040,0.5,1,21.219145,0.2500,NaN,20.989734,21.019431,-7.339601
2,-1,24.929371,0.5,1,24.929473,NaN,0.250,24.412359,24.437156,-7.340415
3,-1,29.252499,0.5,1,29.252608,NaN,0.125,27.680332,27.701763,-7.340949
4,-1,32.474228,0.5,1,32.474352,NaN,0.250,31.983968,32.003166,-7.339941
5,-1,36.108979,0.5,1,36.109089,NaN,1.000,35.751924,35.751458,-7.338652
...,...,...,...,...,...,...,...,...,...,...
639,-1,3568.994364,0.2,1,3568.994498,NaN,0.125,3568.136317,3568.143117,-4.493479
640,-1,3572.878115,0.2,1,3572.878243,NaN,0.125,3572.382293,3572.394658,-4.499667
642,-1,3593.259743,0.2,-1,3593.260771,0.0625,NaN,3592.800117,3592.809983,-4.500962
644,1,3608.220022,0.2,-1,3608.220975,NaN,0.000,3607.904307,3607.924405,-4.502257
